In [ ]:
import os
from datasets import load_from_disk
from tqdm.auto import tqdm, trange
from datasets import load_dataset, load_from_disk
from open_clip import get_tokenizer
from metrics import (
    load_perplexity_model_and_tokenizer,
    compute_prompt_perplexity,
    load_aesthetics_and_artifacts_models,
    compute_aesthetics_and_artifacts_scores,
)


# dataset = load_dataset("poloclub/diffusiondb", "2m_random_100k")

# dataset.save_to_disk(
#     "/fs/nexus-scratch/mcding/Watermark-Experiments/datasets/source/diffusiondb/2m_random_100k"
# )

# dataset = load_from_disk(
#     os.path.join(os.environ.get("DATASET_DIR"), "./source/diffusiondb/2m_random_100k")
# )["train"]

# dataset = dataset.filter(
#     # filter width and height
#     lambda x: (x["width"] == 512 and x["height"] == 512)
#     # filter diffusion hyperparameters
#     and (x["step"] == 50 and x["cfg"] == 7 and x["sampler"] == "k_lms")
#     # filter nsfw
#     and (x["image_nsfw"] < 0.2 and x["prompt_nsfw"] < 0.1)
# )
# dataset = dataset.remove_columns(
#     [
#         "seed",
#         "step",
#         "cfg",
#         "sampler",
#         "width",
#         "height",
#         "user_name",
#         "timestamp",
#         "image_nsfw",
#         "prompt_nsfw",
#     ]
# )

# dataset.save_to_disk(
#     os.path.join(os.environ.get("DATASET_DIR"), "./selected/diffusiondb_temp1")
# )

# dataset = load_from_disk(
#     os.path.join(os.environ.get("DATASET_DIR"), "./selected/diffusiondb_temp1")
# )

# clip_tokenizer = get_tokenizer("ViT-g-14")

# dataset = dataset.filter(
#     lambda data: 0 < len(clip_tokenizer.encode(data["prompt"])) <= 75
# )

# dataset.save_to_disk(
#     os.path.join(os.environ.get("DATASET_DIR"), "./selected/diffusiondb_temp2")
# )

# dataset = load_from_disk(
#     os.path.join(os.environ.get("DATASET_DIR"), "./selected/diffusiondb_temp2")
# )

# prompts = [dataset[i]["prompt"] for i in trange(len(dataset))]

# clip_tokenizer = get_tokenizer("ViT-g-14")

# normalized_prompts = []

# for prompt in tqdm(prompts):
#     normalized_prompts.append(clip_tokenizer.decode(clip_tokenizer.encode(prompt)))

# dataset = dataset.add_column("normalized_prompt", normalized_prompts)

# dataset = dataset.rename_column("prompt", "original_prompt")
# dataset = dataset.rename_column("normalized_prompt", "prompt")
# dataset = dataset.remove_columns(["original_prompt"])

# dataset.save_to_disk(
#      os.path.join(os.environ.get("DATASET_DIR"), "./selected/diffusiondb_temp3")
# )

# dataset = load_from_disk(
#     os.path.join(os.environ.get("DATASET_DIR"), "./selected/diffusiondb_temp3")
# )

# unique_prompts = dataset.unique("prompt")

# prompt_to_index = {}

# for i, prompt in tqdm(enumerate(dataset["prompt"]), total=len(dataset)):
#     if prompt not in prompt_to_index:
#         prompt_to_index[prompt] = i
# dataset = dataset.select(list(prompt_to_index.values()))

# assert len(unique_prompts) == len(dataset)


# ppl_models = load_perplexity_model_and_tokenizer()
# aa_models = load_aesthetics_and_artifacts_models()


# images = [dataset[i]["image"] for i in trange(len(dataset))]

# prompts = [dataset[i]["prompt"] for i in trange(len(dataset))]

# ppls = []
# for prompt in tqdm(prompts):
#     ppls.append(compute_prompt_perplexity(prompt, ppl_models))

# aesthetics = []
# artifacts = []
# for image in tqdm(images):
#     aesthetic, artifact = compute_aesthetics_and_artifacts_scores(image, aa_models)
#     aesthetics.append(aesthetic)
#     artifacts.append(artifact)


# dataset = dataset.add_column("ppl", ppls)
# dataset = dataset.add_column("aesthetic", aesthetics)
# dataset = dataset.add_column("artifacts", artifacts)

dataset = load_from_disk(
    os.path.join(os.environ.get("DATASET_DIR"), "./selected/diffusiondb_temp4")
)

print(dataset)

In [ ]:
import numpy as np

aesthetics = dataset["aesthetic"]
artifacts = dataset["artifacts"]
ppls = dataset["ppl"]

key = np.array(aesthetics) + (10 - np.array(artifacts))

dataset_appended = dataset.add_column("key", key)

dataset_sorted = dataset_appended.sort("key", reverse=True)

In [ ]:
from IPython.display import display
import json

prompt_dict = {}
for i in trange(5000):
    # dataset_sorted[i]["image"].save(f"/fs/nexus-projects/HuangWM/datasets/main/diffusiondb/real/{i}.png")
    prompt_dict[str(i)] = dataset_sorted[i]["prompt"]

with open(
    "/fs/nexus-projects/HuangWM/datasets/main/diffusiondb/prompts_new_utf8.json",
    "w",
) as json_file:
    json.dump(prompt_dict, json_file, ensure_ascii=False, indent=4)

In [ ]:
# dataset.save_to_disk(
#     os.path.join(os.environ.get("DATASET_DIR"), "./selected/diffusiondb_temp5")
# )